## Aggregate locations of dense [AIS broadcast reports](https://marinecadastre.gov/ais) at the port of Miami, FL.

In [9]:
gdb_path = os.path.join('data','Miami.gdb')

In [10]:
df = spark\
    .read \
    .format("gdb") \
    .options(path=gdb_path, name="GDB_SystemCatalog") \
    .load()\
    .show(truncate=False)

+---+-------------------------+----------+
|ID |Name                     |FileFormat|
+---+-------------------------+----------+
|1  |GDB_SystemCatalog        |0         |
|2  |GDB_DBTune               |0         |
|3  |GDB_SpatialRefs          |0         |
|4  |GDB_Items                |0         |
|5  |GDB_ItemRelationships    |0         |
|6  |GDB_ItemRelationshipTypes|0         |
|7  |GDB_ItemTypes            |0         |
|8  |GDB_ReplicaLog           |2         |
|9  |MiamiExtent              |0         |
|10 |Voyage                   |0         |
|11 |Broadcast                |0         |
|12 |Vessel                   |0         |
|13 |BaseStations             |0         |
|14 |AttributeUnits           |0         |
|15 |Extent                   |0         |
+---+-------------------------+----------+



### Create a Spark Dataframe and register it as a table for upcoming SQL operations

In [11]:
df = spark\
    .read \
    .format("com.esri.gdb") \
    .options(path=gdb_path, name="Broadcast") \
    .load()

In [12]:
df.printSchema()

root
 |-- OBJECTID: integer (nullable = false)
 |-- Shape: struct (nullable = true)
 |    |-- x: double (nullable = true)
 |    |-- y: double (nullable = true)
 |-- SOG: integer (nullable = true)
 |-- COG: integer (nullable = true)
 |-- Heading: integer (nullable = true)
 |-- ROT: integer (nullable = true)
 |-- BaseDateTime: timestamp (nullable = true)
 |-- Status: integer (nullable = true)
 |-- VoyageID: integer (nullable = true)
 |-- MMSI: integer (nullable = true)
 |-- ReceiverType: string (nullable = true)
 |-- ReceiverID: string (nullable = true)



In [13]:
df.createOrReplaceTempView("Broadcast")

Create temp table `QR` by mapping a `Broadcast` event to a cell location.  The cell size is 0.001 degrees.

In [14]:
cell_1 = 0.001
cell_2 = cell_1 * 0.5

sql(f"""
select
cast(floor(Shape.x/{cell_1}) as INT) as q,
cast(floor(Shape.y/{cell_1}) as INT) as r
from Broadcast
""")\
    .createOrReplaceTempView("QR")

Aggregate by cell and report back only the cells with more that 100 `Broadcast` events.
Map each cell q/r to an x/y geo location and convert the Spark dataframe into a Pandas dataframe.

In [15]:
sql(f"""
select
q*{cell_1}+{cell_2} as x,
r*{cell_1}+{cell_2} as y,
count(1) as pop
from QR
group by q,r
having pop > 100
""")\
    .show()

+--------+-------+-----+
|       x|      y|  pop|
+--------+-------+-----+
|-80.2455|25.7965|12737|
|-80.1505|25.7655|  240|
|-80.1265|25.7605|  201|
|-80.0845|25.7935| 5279|
|-80.1605|25.7735|17623|
|-80.1485|25.7655| 1600|
|-80.1645|25.7665|  136|
|-80.1665|25.7755| 2935|
|-80.1535|25.7665| 2823|
|-80.1785|25.7805| 2093|
|-80.1495|25.7665|  237|
|-80.2465|25.7975|22725|
|-80.2515|25.8015|15007|
|-80.1685|25.7705| 4349|
|-80.1655|25.7675|  160|
|-80.1605|25.7655|  126|
|-80.1765|25.7815|  160|
|-80.1425|25.7665|  190|
|-80.1055|25.7595|  144|
|-80.2145|25.7815|  177|
+--------+-------+-----+
only showing top 20 rows

